# X ray Image Classification

## Table of content
<ul>
    <li><a href="#dr">Data Reading.</a></li>
    <li><a href="#i">Import Packages.</a></li>
    <li><a href="#dp">Data Pre-processing.</a></li>
    <li><a href="#dl">Deep Learning Model.</a></li>
    <li><a href="#s">Model Saving.</a></li>
    <li><a href="#mt">Model Testing.</a></li>
    </ul>

<a id="dr"></a>
## Data Reading

In [1]:
! mkdir data
! unzip /content/Dataset.zip -d data

Archive:  /content/Dataset.zip
   creating: data/Dataset/
   creating: data/Dataset/COVID PATIENTS/
  inflating: data/Dataset/COVID PATIENTS/covid_0.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_1.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_10.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_100.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_101.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_102.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_103.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_104.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_105.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_106.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_107.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_108.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_109.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_11.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_110.jpg  
  inflating: data/Dataset/COVID PATIENTS/covid_111.

<a id="i"></a>
## Import Packages

In [2]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.metrics import AUC, TruePositives, TrueNegatives, FalsePositives, FalseNegatives
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

<a id="dp"></a>
## Data Pre-processing

In [3]:
# Create a data generator                             
train_datagen = ImageDataGenerator(rotation_range=10,
                                   validation_split = 0.2,
                                       rescale=1./255,
                              horizontal_flip=True)

In [4]:
# load and iterate training dataset
train = train_datagen.flow_from_directory(
    "/content/data/Dataset",
    target_size=(224, 224),
    class_mode='binary',
    color_mode = 'grayscale',
    batch_size=32,
    subset='training'
)

valid =train_datagen.flow_from_directory(
    "/content/data/Dataset",
    target_size=(224, 224),
    class_mode='binary',
    color_mode = 'grayscale',
    batch_size=32,
    subset='validation'
)

Found 260 images belonging to 2 classes.
Found 64 images belonging to 2 classes.


<a id="dl"></a>
## Deep Learning Model

In [5]:
Model = keras.models.Sequential()

Model.add(keras.layers.Conv2D(200,kernel_size=(3,3),activation='relu'))
Model.add(keras.layers.MaxPool2D(4,4))
Model.add(keras.layers.Conv2D(120,kernel_size=(3,3),activation='relu')) 
Model.add(keras.layers.Conv2D(50,kernel_size=(3,3),activation='relu'))
Model.add(keras.layers.MaxPool2D(4,4))
Model.add(keras.layers.Flatten())
Model.add(keras.layers.Dense(120,activation='relu'))
Model.add(keras.layers.Dense(50,activation='relu'))  
Model.add(keras.layers.Dropout(rate=0.5))
Model.add(keras.layers.Dense(1,activation='sigmoid'))

In [6]:
epochs = 30
lr = 1e-4

Model.compile(optimizer=Adam(learning_rate=lr), loss='binary_crossentropy', metrics='accuracy')

In [15]:
Model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, None, None, 200)   2000      
                                                                 
 max_pooling2d (MaxPooling2D  (None, None, None, 200)  0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 120)   216120    
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 50)    54050     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, None, None, 50)   0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, None)              0

In [7]:
Model_history = Model.fit(train,
                                epochs=epochs,
                                verbose=1,
                                validation_data=valid,
                                steps_per_epoch=train.samples/train.batch_size,
                                validation_steps=valid.samples/valid.batch_size)

Epoch 1/30
8/8 [==============================] - 54s 7s/step - loss: 0.6927 - accuracy: 0.5038 - val_loss: 0.6902 - val_accuracy: 0.5000
Epoch 2/30
8/8 [==============================] - 52s 7s/step - loss: 0.6930 - accuracy: 0.4923 - val_loss: 0.6869 - val_accuracy: 0.5000
Epoch 3/30
8/8 [==============================] - 52s 6s/step - loss: 0.6848 - accuracy: 0.5731 - val_loss: 0.6845 - val_accuracy: 0.5469
Epoch 4/30
8/8 [==============================] - 53s 7s/step - loss: 0.6822 - accuracy: 0.5923 - val_loss: 0.6787 - val_accuracy: 0.5938
Epoch 5/30
8/8 [==============================] - 52s 6s/step - loss: 0.6707 - accuracy: 0.6962 - val_loss: 0.6655 - val_accuracy: 0.7656
Epoch 6/30
8/8 [==============================] - 55s 7s/step - loss: 0.6482 - accuracy: 0.6769 - val_loss: 0.6480 - val_accuracy: 0.6094
Epoch 7/30
8/8 [==============================] - 53s 7s/step - loss: 0.6222 - accuracy: 0.6808 - val_loss: 0.6124 - val_accuracy: 0.7344
Epoch 8/30
8/8 [==================

<a id="s"></a>
## Model Saving

In [8]:
Model.save("image_classification.h5")

<a id="mt"></a>
## Model Testing

In [9]:
Model.evaluate(train)

9/9 [==============================] - 13s 1s/step - loss: 0.2529 - accuracy: 0.8962


[0.252910315990448, 0.8961538672447205]

In [10]:
Model.evaluate(valid)

2/2 [==============================] - 3s 2s/step - loss: 0.3670 - accuracy: 0.8750


[0.3669748306274414, 0.875]

### Test on COVID IMAGE

In [12]:
my_image = load_img("/content/data/Dataset/COVID PATIENTS/covid_13.jpg", target_size=(224, 224), grayscale=True)
my_image = img_to_array(my_image)
my_image = my_image.reshape(1, my_image.shape[0], my_image.shape[1], my_image.shape[2])
Model.predict(my_image)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


array([[0.]], dtype=float32)

### Test on Normal IMAGE

In [14]:
my_image = load_img("/content/data/Dataset/NORMAL PATIENTS/normal_0.jpg", target_size=(224, 224), grayscale=True)
my_image = img_to_array(my_image)
my_image = my_image.reshape(1, my_image.shape[0], my_image.shape[1], my_image.shape[2])
Model.predict(my_image)

/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


array([[1.]], dtype=float32)